In [406]:
allure_etudier=1

In [407]:
import  sys

#LOCALISATION DES DONNEES
sys.path.insert(0, "C:/projets_python/diabolo")

import warnings
warnings.filterwarnings("ignore")



#LIBRAIRIES PERSO
import etude_variable.MyLog as log
#import etude_variable.jouer as jouer

#import etude_variable.analyse as ana

# LIBRAIRIE PYHTON CLASSIQUES
import pandas as pd
import numpy as np
import matplotlib
import scipy
import platform

from collections import Counter

#ESTIMATEUR
import sklearn
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.ensemble import  RandomForestClassifier
from sklearn import neighbors, datasets
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Imputer

from sklearn.ensemble import GradientBoostingClassifier

#MISE A l ECHELLE
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import quantile_transform

from scipy.stats import uniform
from scipy.stats import randint

#TRAINING
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from skopt import BayesSearchCV
import dask_searchcv as dcv
from skopt.space import Real, Integer
from sklearn.model_selection import cross_validate

#Evaluateur
from sklearn.model_selection import KFold

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold

from sklearn.feature_selection import SelectFromModel


from evolutionary_search import EvolutionaryAlgorithmSearchCV
from sklearn.utils import class_weight

#outils
from dask.diagnostics import ProgressBar
import logging
from datetime import datetime


#Metriques
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import learning_curve
from sklearn import  metrics
from sklearn.metrics import  roc_auc_score
from sklearn.metrics import  classification_report

#Outils

import time


#Graphique
from matplotlib import pyplot
from matplotlib import pyplot as plt
import seaborn as sns

print('Operating system version....', platform.platform())
print("Python version is........... %s.%s.%s" % sys.version_info[:3])
print('scikit-learn version is.....', sklearn.__version__)
print('pandas version is...........', pd.__version__)
print('numpy version is............', np.__version__)
print('matplotlib version is.......', matplotlib.__version__)
print('scipy version is.......', scipy.__version__)


Operating system version.... Windows-10-10.0.17134-SP0
Python version is........... 3.6.5
scikit-learn version is..... 0.19.1
pandas version is........... 0.22.0
numpy version is............ 1.14.2
matplotlib version is....... 2.2.0
scipy version is....... 1.0.0


# LECTURE DES DONNEES

In [408]:
def lecture_data(Fichier, xnames, xindex_col, allure=1, mode_debug=0, avec_index=True):
    
    print("DATA ....")
    start_time=timer()
    if avec_index==True:
        df = pd.read_csv(Fichier,  index_col=xindex_col,     sep=';',     names=xnames,               skipinitialspace=True,              encoding='Latin-1')
    else:
        df = pd.read_csv(Fichier,   index_col=None,  sep=';',                     names=xnames,                         skipinitialspace=True,     encoding='Latin-1')

    df = df.groupby("ALLURE")
    df = df.get_group(allure)
    #print(df.info())

    print(timer(start_time))
    return df


In [409]:
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        print(start_time)
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))
        
        


In [410]:
def evaluation_2(model, train_x, train_y, test_x, test_y):    
    from sklearn.feature_selection import RFE
    rfc=model
    # create the RFE model and select 4 attributes
    rfe_model = RFE(rfc, 9, step=1)
    rfe_model = rfe_model.fit(train_x, train_y)
    # summarize the selection of the attributes
    print(rfe_model.support_)
    print(rfe_model.ranking_)

    # evaluate the model on testing set
    pred_y = rfe_model.predict(test_x)
    predictions = [round(value) for value in pred_y]
    accuracy = accuracy_score(test_y, predictions)
    print("Test Accuracy: %.2f%%" % (accuracy*100.0))
    print(train_x.mean()   )

# fit speciaux

In [411]:
def my_fit2(estimateur, train_x, train_y):
    kfold = StratifiedKFold(n_splits=10, random_state=None, shuffle=False)
    
    results = cross_val_score(estimateur, train_x, train_y, cv=kfold)
    print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
    
    y_pred = model.predict(test_x)
    predictions = [round(value) for value in y_pred]
    return model,y_pred,predictions


def fit_special(estimateur,    test_x,test_y, train_x, train_y):
    # Use SelectFromModel
    thresholds = np.sort(estimateur.best_estimator_.named_steps["clf"].feature_importances_)

    for thresh in thresholds:
        # select features using threshold
        selection = SelectFromModel(estimateur, threshold=thresh, prefit=True)
        select_X_train = selection.transform(train_x)

        # train model
        selection_model = estimateur
        selection_model.fit(select_X_train, train_y)

        # eval model
        select_X_test = selection.transform(test_x)
        y_pred = selection_model.predict(select_X_test)
        predictions = [round(value) for value in y_pred]
        accuracy = accuracy_score(test_y, predictions)
        print("Thresh=%.3f, n=%d, Accuracy: %.2f%%" % (thresh, select_X_train.shape[1], accuracy * 100.0))
        print(confusion_matrix(test_y, predictions))
        print(classification_report(test_y, predictions))

    y_pred = estimateur.predict(test_x)
    predictions = [round(value) for value in y_pred]
    return estimateur,y_pred,predictions



In [412]:
def mise_en_relation(df,crit,n):
  
    print(df_course)
    qt=quantile_transform(df.loc[df['IDCOURSE']==n, crit], n_quantiles=20, random_state=0,subsample=25 )
    
    
    return qt


In [413]:
def sclalerise_course(df):
    print("Scalerise_course ...")
    start_time=timer()
    for n, g in df.groupby('IDCOURSE'):
        #df.loc[df['IDCOURSE'] ==n, 'PAR_REUSSITE_3P_'] = quantile_transform(df.loc[df['IDCOURSE']==n, ['PAR_REUSSITE_3P']], n_quantiles=20, random_state=0,subsample=25 )
        df.loc[df['IDCOURSE'] ==n, 'PAR_REUSSITE_3P_'] = mise_en_relation(df,['PAR_REUSSITE_3P'],n)
        df.loc[df['IDCOURSE'] ==n, 'PAR_CARRIERE_'] = mise_en_relation(df,['PAR_CARRIERE'],n)
        df.loc[df['IDCOURSE'] ==n, 'PAR_CARRIERE_Q_'] = mise_en_relation(df,['PAR_CARRIERE_Q'],n)
        df.loc[df['IDCOURSE'] ==n, 'PAR_CLASSE_AGE_'] = mise_en_relation(df,['PAR_CLASSE_AGE'],n)
        df.loc[df['IDCOURSE'] ==n, 'PAR_COTEDER_'] = mise_en_relation(df,['PAR_COTEDER'],n)
        df.loc[df['IDCOURSE'] ==n, 'PAR_ENT_ECART_GAGNANT_'] = mise_en_relation(df,['PAR_ENT_ECART_GAGNANT'],n)
        df.loc[df['IDCOURSE'] ==n, 'PAR_ENT_RAPPORT_GAGNANT_M_'] = mise_en_relation(df,['PAR_ENT_RAPPORT_GAGNANT_M'],n)    
        df.loc[df['IDCOURSE'] ==n, 'PAR_ENT_REU_PLACE_'] = mise_en_relation(df,['PAR_ENT_REU_PLACE'],n)    
        df.loc[df['IDCOURSE'] ==n, 'PAR_ENT_REUSSITE_GAGNE_'] = mise_en_relation(df,['PAR_ENT_REUSSITE_GAGNE'],n)    
        df.loc[df['IDCOURSE'] ==n, 'PAR_ENT_VICTOIRE_'] = mise_en_relation(df,['PAR_ENT_VICTOIRE'],n)    
        df.loc[df['IDCOURSE'] ==n, 'PAR_PLACE_'] = mise_en_relation(df,['PAR_PLACE'],n)  
        df.loc[df['IDCOURSE'] ==n, 'PAR_PLACE_Q_'] = mise_en_relation(df,['PAR_PLACE_Q'],n)  
        df.loc[df['IDCOURSE'] ==n, 'PAR_REUSSITE_GAGNE_'] = mise_en_relation(df,['PAR_REUSSITE_GAGNE'],n)           
        df.loc[df['IDCOURSE'] ==n, 'PAR_REUSSITE_QUINTE_'] = mise_en_relation(df,['PAR_REUSSITE_QUINTE'],n)          
        df.loc[df['IDCOURSE'] ==n, 'PAR_RUESSITE_PLACE_'] = mise_en_relation(df,['PAR_RUESSITE_PLACE'],n)  
        df.loc[df['IDCOURSE'] ==n, 'POIDS_'] = mise_en_relation(df,['POIDS'],n) 
        df.loc[df['IDCOURSE'] ==n, 'PAR_ENT_ECART_PLACE_'] = mise_en_relation(df,['PAR_ENT_ECART_PLACE'],n) 
        df.loc[df['IDCOURSE'] ==n, 'PAR_VICTOIRE_'] = mise_en_relation(df,['PAR_VICTOIRE'],n) 
        df.loc[df['IDCOURSE'] ==n, 'PAR_VICTOIRE_Q_'] = mise_en_relation(df,['PAR_VICTOIRE_Q'],n) 
        df.loc[df['IDCOURSE'] ==n, 'PAR_ENT_NB_COURSE_'] = mise_en_relation(df,['PAR_ENT_NB_COURSE'],n) 
        
        
    df[['PAR_REUSSITE_3P']] = df[['PAR_REUSSITE_3P_']]
    df[['PAR_CARRIERE']]= df[['PAR_CARRIERE_']]
    df[['PAR_CARRIERE_Q']] = df[['PAR_CARRIERE_Q_']]
    df[['PAR_CLASSE_AGE']] = df[['PAR_CLASSE_AGE_']]
    df[['PAR_COTEDER']] = df[['PAR_COTEDER_']]
    df[['PAR_ENT_ECART_GAGNANT']] = df[['PAR_ENT_ECART_GAGNANT_']]
    df[['PAR_ENT_RAPPORT_GAGNANT_M']] = df[['PAR_ENT_RAPPORT_GAGNANT_M_']]
    df[['PAR_ENT_REU_PLACE']] = df[['PAR_ENT_REU_PLACE_']]
    df[['PAR_ENT_REUSSITE_GAGNE']] = df[['PAR_ENT_REUSSITE_GAGNE_']]
    df[['PAR_ENT_VICTOIRE']]= df[['PAR_ENT_VICTOIRE_']]
    df[['PAR_PLACE']]= df[['PAR_PLACE_']]
    df[['PAR_PLACE_Q']]= df[['PAR_PLACE_Q_']]
    df[['PAR_REUSSITE_GAGNE']]= df[['PAR_REUSSITE_GAGNE_']]
    df[['PAR_REUSSITE_QUINTE']]= df[['PAR_REUSSITE_QUINTE_']]
    df[['PAR_RUESSITE_PLACE']]= df[['PAR_RUESSITE_PLACE_']]
    df[['POIDS']]= df[['POIDS_']]
    df[['PAR_ENT_ECART_PLACE']]= df[['PAR_ENT_ECART_PLACE_']]
    df[['PAR_VICTOIRE']]= df[['PAR_VICTOIRE_']]
    df[['PAR_VICTOIRE_Q']]= df[['PAR_VICTOIRE_Q_']]
    df[['PAR_ENT_NB_COURSE']]= df[['PAR_ENT_NB_COURSE_']]
    
    
    print(timer(start_time))
    
    
    
    return df

    

            

In [414]:

def my_drop(df, col):
    if col in df:
        df.drop([col], axis=1, inplace=True)
    return df




def encodage(df_x):
    
    from sklearn import preprocessing
    #le = preprocessing.LabelEncoder()
    #df_gagnant[['HIPPO']] = le.fit_transform(df_gagnant[['HIPPO']])
    #df_gagnant[['PAR_PROPRIO']] = le.fit_transform(df_gagnant[['PAR_PROPRIO']])
    ##f_gagnant[['NOM_JOC']] = le.fit_transform(df_gagnant[['NOM_JOC']])
    #df_gagnant[['NOM_ENTR']] = le.fit_transform(df_gagnant[['NOM_ENTR']])
    #df_gagnant[['CHEVAL']] = le.fit_transform(df_gagnant[['CHEVAL']])
    #df_gagnant[['PAR_NUM']] = le.fit_transform(df_gagnant[['PAR_NUM']])
   # df_gagnant[['MUSIC_CHEVAL']] = le.fit_transform(df_gagnant[['MUSIC_CHEVAL']])








    #scaler = RobustScaler()
   # scaler = RobustScaler( with_scaling=False, quantile_range=(30.0, 70.0), copy=False)

    critere_scale = [
      
              
              'PAR_AGE',
              'PAR_CARRIERE',
              'PAR_CARRIERE_Q',
              'PAR_CLASSE_AGE',
              'PAR_COTEDER',
              'PAR_ENT_ECART_GAGNANT',
              'PAR_ENT_RAPPORT_GAGNANT_M',
              'PAR_ENT_REU_PLACE',
              'PAR_ENT_REUSSITE_GAGNE',
              'PAR_ENT_VICTOIRE',
              'PAR_GAIN',
              'pAR_JOC_ECART_GAGNANT',
              'PAR_JOC_ECART_PLACE',
              'PAR_JOC_NB_COURSE',
              'PAR_JOC_PLACE_3P',
              'pAR_JOC_RAPPORT_GAGNANT_M',
              'PAR_JOC_REU_PLACE',
              'pAR_JOC_REUSSITE_GAGNE',
              'pAR_JOC_VICTOIRE',
              'PAR_NP',
              'PAR_NUM',
              'PAR_PLACE',
              'PAR_PLACE_Q',
              'PAR_REUSSITE_3P',
              'PAR_REUSSITE_GAGNE',
              'PAR_REUSSITE_QUINTE',
              'PAR_RUESSITE_PLACE',

              

          #    'Point',
              

              'POIDS',
              
              
              'PAR_ENT_ECART_PLACE',
              'PAR_VICTOIRE',
              'PAR_VICTOIRE_Q',
          'PAR_ENT_NB_COURSE',
                      ]
    
    

     

    #df_gagnant[critere_scale] = quantile_transform(df_gagnant[critere_scale], n_quantiles=4)

    #df_gagnant[critere_scale] = qt.fit_transform(df_gagnant[critere_scale])
    df_x[critere_scale] = quantile_transform(df_x[critere_scale], n_quantiles=5000, random_state=0,subsample=300000)

   # df_x[critere_scale] = scaler.fit_transform(df_x[critere_scale])

    return df_x,critere_scale


# selection

In [424]:
  # Lecture des données sans index
xnames = ['ALLURE', 'CO_DISTANCE',
              'CO_PRIX', 'HIPPO',
              'IDCOURSE', 'IDPARTCIPANT',
              'PAR_AGE', 'PAR_ARRIVE',
              'PAR_CARRIERE',
              'PAR_CARRIERE_Q',
              'PAR_CLASSE_AGE',
              'PAR_COTEDER',
              'PAR_ENT_ECART_GAGNANT',
              'PAR_ENT_RAPPORT_GAGNANT_M',
              'PAR_ENT_REU_PLACE',
              'PAR_ENT_REUSSITE_GAGNE',
              'PAR_ENT_VICTOIRE',
              'PAR_GAIN',
              'pAR_JOC_ECART_GAGNANT',
              'PAR_JOC_ECART_PLACE',
              'PAR_JOC_NB_COURSE',
              'PAR_JOC_PLACE_3P',
              'pAR_JOC_RAPPORT_GAGNANT_M',
              'PAR_JOC_REU_PLACE',
              'pAR_JOC_REUSSITE_GAGNE',
              'pAR_JOC_VICTOIRE',
              'PAR_NP',
              'PAR_NUM',
              'PAR_PLACE',
              'PAR_PLACE_Q',
              'PAR_REUSSITE_3P',
              'PAR_REUSSITE_GAGNE',
              'PAR_REUSSITE_QUINTE',
              'PAR_RUESSITE_PLACE',
              'autostart',
              'cendre',
              'grande_piste',
              'Point',
              'Nb_partant',
              'PAR_PROPRIO',
              'NOM_JOC',
              'NOM_ENTR',
              'POIDS',
              'CORDE', 'CHEVAL',
              'MUSIC_CHEVAL',
              'MUSIC_ENT',
              'MUSIC_JOC',
              'PAR_VALEUR',
              'PAR_ENT_ECART_PLACE',
              'PAR_VICTOIRE',
              'PAR_VICTOIRE_Q',
          'PAR_ENT_NB_COURSE',
              'FIN_ligne']

# Creating bins for the win column
def assign_selection(W):
    if W <=2:
        return 1
    else:
        return 0
    
    # Creating bins for the win column
def assign_selection2(W):
    if W == 1.0:
        return 1
    if W == 0.0:
        return 0




    

In [434]:
index_col = ['IDCOURSE','IDPARTCIPANT']
start_time=timer()
df = lecture_data('d:\data_diabolo.csv', xnames, xindex_col=index_col, allure=allure_etudier, avec_index=False)
df.reindex(columns = ['IDCOURSE', 'IDPARTCIPANT']) 
print(timer(start_time))

2018-06-24 22:47:18.000190
DATA ....
2018-06-24 22:47:18.001188

 Time taken: 0 hours 0 minutes and 2.44 seconds.
None

 Time taken: 0 hours 0 minutes and 2.44 seconds.
None


In [435]:
df['MAXX']=0
df['CRI']=0
df['POINT2018']=0

def donne_note(val_cri, maxx):
    if maxx==0:
        return 0
    else:
        return (val_cri/maxx)*20
    
    
def calcul_note(df, num_course, cri):

    print(cri)
    dd=df.loc[df['IDCOURSE']==num_course,cri]  # selection critere
    #determine max de ce critere
    maxx=dd.max() #MAXI
    print(maxx)
    df.loc[df['IDCOURSE']==num_course,'MAXX']=maxx[0] #selection Max sans colonne

    #print( num_course, (df.loc[df['IDCOURSE']==num_course,  cri[0]] ))

    df.loc[df['IDCOURSE']==num_course,'CRI'] =  ( df.loc[df['IDCOURSE']==num_course,  cri[0]]   / maxx[0])*20
    df.loc[df['IDCOURSE']==num_course, cri[0]] =  df.loc[df['IDCOURSE']==num_course,'CRI'] 
        
    
  
   #df.loc[df['IDCOURSE']==ncourse,'POINT2018'] = df.loc[df['IDCOURSE']==ncourse,'POINT2018'] + df.loc[df['IDCOURSE']==ncourse,'CRI']
    return  df



      


In [437]:
i=0
for n  in df.groupby(['IDCOURSE'],axis=0):
    nc=n[0]
    print("Course ",nc)
    df= calcul_note(df, nc, ['PAR_REUSSITE_3P'] )
    #print("NOTE  **********df=",df.loc[df['IDCOURSE']==ncourse,'PAR_REUSSITE_3P'])
    
    #Somme de point obtenu pour UN criter pour le course NC
    df.loc[df['IDCOURSE']==nc,'POINT2018']    =   df.loc[df['IDCOURSE']==nc,'POINT2018'] + df.loc[df['IDCOURSE']==nc,'CRI']
    print('\n',df.loc[df['IDCOURSE']==nc,'POINT2018'] )
          
    i=i+1
    if i==4:
        break
        
        
    

Course  1616
['PAR_REUSSITE_3P']
PAR_REUSSITE_3P    0.533333
dtype: float64

 13    14.634159
14     4.918053
15    11.295195
16    12.499995
17    11.132819
18    10.227269
19    11.383920
20    12.499995
21    16.216210
22    14.880947
23     8.388155
24    10.500007
25    20.000000
26     9.740256
27     6.976767
28    12.499995
29    17.500023
30     7.870392
Name: POINT2018, dtype: float64
Course  1617
['PAR_REUSSITE_3P']
PAR_REUSSITE_3P    0.545455
dtype: float64

 31    16.666673
32    16.041653
33    19.743590
34     8.461523
35    16.666673
36     7.638870
37    11.578957
38    12.222200
39    12.222200
40    20.000000
41     9.777782
42     9.166659
Name: POINT2018, dtype: float64
Course  1618
['PAR_REUSSITE_3P']
PAR_REUSSITE_3P    0.571429
dtype: float64

 43    14.736844
44    14.259234
45    11.199992
46    12.173901
47    20.000000
48    14.736844
49    13.333310
50     7.954549
51    12.894725
52     4.374997
53    13.999990
54    12.962940
55    17.499987
56     9.84374